# Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Load Data

In [ ]:
#sample_submission = pd.read_csv("../input/reducing-commercial-aviation-fatalities/sample_submission.csv")
#test = pd.read_csv("../input/reducing-commercial-aviation-fatalities/test.csv")
train = pd.read_csv("../input/reducing-commercial-aviation-fatalities/train.csv")

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [ ]:
#train = reduce_mem_usage(train)
#test = reduce_mem_usage(test)

In [ ]:
#test.head()

In [ ]:
train.head()

In [ ]:
print(train.shape)

In [ ]:
pd.crosstab(train.experiment, train.event)

In [ ]:
pd.crosstab(train.experiment, train.crew)

In [ ]:
pd.crosstab(train.experiment, train.seat)

In [ ]:
print(list(enumerate(train.columns)))

In [ ]:
crew = 3
seat = 0
exp = 'DA'
ev = 'D'

sel = (train.crew == crew) & (train.experiment == exp) & (train.seat == seat)
pilot_info = train.loc[sel,:].sort_values(by='time')


plt.figure(figsize=[16,12])
for i in range(4, 27):
    plt.subplot(6,4,i-3)
    plt.plot(pilot_info.time, 
             pilot_info.iloc[:,i], zorder=1)
    plt.scatter(pilot_info.loc[pilot_info.event ==  ev,:].time, 
             pilot_info.loc[pilot_info.event == ev,:].iloc[:,i], c='red', zorder=2, s=1)

plt.show()

# Create Feature and Label Arrays

In [ ]:
y_train = train.event
X_train = train.iloc[:,4:27]
X_train.head()

In [ ]:
pd.DataFrame({
    'min_val':X_train.min(axis=0).values,
    'max_val':X_train.max(axis=0).values
}, index = X_train.columns
)

In [ ]:
y_train.value_counts() 

In [ ]:
y_train.value_counts() / len(y_train)

# Sample Training Set for Faster Training

In [ ]:
#X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.98, stratify=y_train_full, random_state=1)

#print(X_train.shape)

## Random Forest

In [ ]:
%%time 

rf_mod = RandomForestClassifier(n_estimators=10, max_depth=32, n_jobs=-1)
rf_mod.fit(X_train, y_train)

print('Training Accuracy:  ', rf_mod.score(X_train, y_train))
print('Validation Accuracy:', rf_mod.score(X_valid, y_valid))

In [ ]:
%%time
rf_pipe = Pipeline(
    steps = [
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=10, n_jobs=-1))
    ]
)

lr_param_grid = {
    'classifier__max_depth': [8, 16, 32, 64]
}


np.random.seed(1)
grid_search = GridSearchCV(rf_pipe, lr_param_grid, cv=5, refit='True', verbose = 10)
grid_search.fit(X_train, y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
grid_search.cv_results_['mean_test_score']

In [ ]:
%%time 
rf_mod = RandomForestClassifier(n_estimators=100, max_depth=32, n_jobs=-1)
rf_mod.fit(X_train, y_train)

print('Training Accuracy:  ', rf_mod.score(X_train, y_train))
print('Validation Accuracy:', rf_mod.score(X_valid, y_valid))

In [ ]:
rf_mod.predict_proba(X_train)

In [ ]:
from sklearn.metrics import log_loss

log_loss(y_train, rf_mod.predict_proba(X_train))

In [ ]:
log_loss(y_valid, rf_mod.predict_proba(X_valid))

Score: 0.61190

# Gradient Boosting Tree

In [ ]:
%%time 

xbg_mod = XGBClassifier()
xbg_mod.fit(X_train, y_train)

xbg_mod.score(X_train, y_train)

In [ ]:
xbg_mod.score(X_valid, y_valid)

In [ ]:
log_loss(y_train, xbg_mod.predict_proba(X_train))

In [ ]:
log_loss(y_valid, xbg_mod.predict_proba(X_valid))

# Generate Test Predictions

In [ ]:
test_iterator = pd.read_csv('../input/reducing-commercial-aviation-fatalities/test.csv', chunksize=5)
test_top = next(test_iterator)
test_top

In [ ]:
print(xbg_mod.predict_proba(test_top.iloc[:,5:]))

In [ ]:
%%time 

cs = 1000000
i = 0

for test in pd.read_csv('../input/reducing-commercial-aviation-fatalities/test.csv', chunksize=cs):
  
    print('--Iteration',i, 'is started')
    
    test_pred = xbg_mod.predict_proba(test.iloc[:,5:])
    
    partial_submission = pd.DataFrame({
        'id':test.id,
        'A':test_pred[:,0],
        'B':test_pred[:,1],
        'C':test_pred[:,2],
        'D':test_pred[:,3]
    })
        
    if i == 0:
        submission = partial_submission.copy()
    else:
        submission = submission.append(partial_submission, ignore_index=True)
        

    del test
    print('++Iteration', i, 'is done!')
    i +=1

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)